In [3]:
!git clone --depth=1 https://github.com/twintproject/twint.git
%cd twint
!pip3 install . -r requirements.txt

Cloning into 'twint'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 47 (delta 3), reused 14 (delta 0), pack-reused 0
Unpacking objects: 100% (47/47), done.
/content/twint
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/twint
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 263 kB 7.5 MB/s 
     |████████████████████████████████| 384 kB 47.0 MB/s 
     |████████████████████████████████| 288 kB 43.2 MB/s 
     |█████████████████████████

In [4]:
!pip install aiohttp==3.7.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 6.8 MB/s 
  Attempting uninstall: async-timeout
    Found existing installation: async-timeout 4.0.2
    Uninstalling async-timeout-4.0.2:
      Successfully uninstalled async-timeout-4.0.2
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.8.1
    Uninstalling aiohttp-3.8.1:
      Successfully uninstalled aiohttp-3.8.1


In [5]:
pip install nest_asyncio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import nest_asyncio
nest_asyncio.apply()
import twint
import pandas as pd
import numpy as np
import re
import os

In [7]:
c = twint.Config()
c.Username = 'ATCS_DISHUB_DIY'
c.Pandas = True
#c.Limit = 10
twint.run.Search(c)

1576775407927382017 2022-10-03 03:25:26 +0000 <ATCS_DISHUB_DIY> 10.24 WIB. Kondisi lalin di Simpang Bandara Adisucipto arus kendaraan terpantau LANCAR dari semua arah/kaki, patuhi rambu lalu lintas dan marka jalan  https://t.co/xph53IJH8l
1576774854686445569 2022-10-03 03:23:15 +0000 <ATCS_DISHUB_DIY> 10.22 WIB. Kondisi lalin di Simpang Maguwo terpantau RAMAI dari arah Barat dan Timur, dari arah Utara terpantau LANCAR  https://t.co/wnul9RBkTU
1576741566886313984 2022-10-03 01:10:58 +0000 <ATCS_DISHUB_DIY> 08.10WIB Arus lalu lintas simpang MMUGM terpantau lancar  https://t.co/HVQq4SDqfs
1576741470438322176 2022-10-03 01:10:35 +0000 <ATCS_DISHUB_DIY> 08.10WIB Arus lalu lintas simpang Demangan terpantau padat  https://t.co/K1bWmslrcc
1576741368919388161 2022-10-03 01:10:11 +0000 <ATCS_DISHUB_DIY> 08.09WIB Arus lalu lintas simpang UIN padat  https://t.co/JiCIEt3Fdn
1576740143805128706 2022-10-03 01:05:19 +0000 <ATCS_DISHUB_DIY> 08.04WIB Arus lalu lintas simpang Kentungan lancar  https://t.

In [8]:
Tweets_df = twint.storage.panda.Tweets_df
#Tweets_df.head()

In [9]:
Tweets_df = Tweets_df.drop(['conversation_id', 'created_at', 'timezone', 'id', 'place', 'language', 'hashtags', 'cashtags', 'user_id', 'user_id_str', 'username', 'name', 'day', 'hour', 'urls', 'video', 'thumbnail', 'retweet', 'nlikes', 'nreplies', 'nretweets', 'quote_url', 'search', 'near', 'geo', 'source', 'user_rt_id', 'user_rt', 'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src', 'trans_dest'], axis = 1)
Tweets_df = Tweets_df.dropna(axis = 1)

In [10]:
Tweets_df.to_excel('CrawlTwint.xlsx', index = False)

In [11]:
#from google.colab import files
#files.download("CrawlTwint.xlsx")

In [12]:
Tweets_df['date'] = Tweets_df['date'].str.replace(r'\D', '', case=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [13]:
print(Tweets_df['date'])

0      20221003032526
1      20221003032315
2      20221003011058
3      20221003011035
4      20221003011011
            ...      
115    20220924081355
116    20220924055620
117    20220924055248
118    20220924002614
119    20220924002552
Name: date, Length: 120, dtype: object


In [14]:
convString = pd.to_datetime(Tweets_df['date'], format='%Y%m%d%H%M%S')

In [15]:
print(convString)

0     2022-10-03 03:25:26
1     2022-10-03 03:23:15
2     2022-10-03 01:10:58
3     2022-10-03 01:10:35
4     2022-10-03 01:10:11
              ...        
115   2022-09-24 08:13:55
116   2022-09-24 05:56:20
117   2022-09-24 05:52:48
118   2022-09-24 00:26:14
119   2022-09-24 00:25:52
Name: date, Length: 120, dtype: datetime64[ns]


In [16]:
Tweets_df['date'] = convString.dt.tz_localize('UTC').dt.tz_convert('Asia/Jakarta')

In [17]:
print(Tweets_df['date'])

0     2022-10-03 10:25:26+07:00
1     2022-10-03 10:23:15+07:00
2     2022-10-03 08:10:58+07:00
3     2022-10-03 08:10:35+07:00
4     2022-10-03 08:10:11+07:00
                 ...           
115   2022-09-24 15:13:55+07:00
116   2022-09-24 12:56:20+07:00
117   2022-09-24 12:52:48+07:00
118   2022-09-24 07:26:14+07:00
119   2022-09-24 07:25:52+07:00
Name: date, Length: 120, dtype: datetime64[ns, Asia/Jakarta]


In [18]:
Tweets_df['date'] = Tweets_df['date'].dt.tz_localize(None)
Tweets_df['date'] = Tweets_df['date'].astype(str)
Tweets_df.to_excel('TwintTimeCleaning.xlsx', index = False)

In [19]:
#from google.colab import files
#files.download("TwintTimeCleaning.xlsx")

In [20]:
key_value = ["jalanan",
    "lalin",
    "lalu lintas",
    "lancar",
    "macet",
    "padat",
    "ramai",
    "tersendat"]
df = pd.read_excel('TwintTimeCleaning.xlsx')
df = df[df['tweet'].str.contains('|'. join(key_value))]
df['tweet'] = df['tweet'].str.replace('http\S+|www.\S+', '', case=False)
df['tweet'] = df['tweet'].str.replace('&amp;', 'dan', case=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  # This is added back by InteractiveShellApp.init_path()


In [21]:
df

,date,tweet,link,photos
0,2022-10-03 10:25:26,10.24 WIB. Kondisi lalin di Simpang Bandara Ad...,https://twitter.com/ATCS_DISHUB_DIY/status/157...,['https://pbs.twimg.com/media/FeHUn8JagAAzAcg....
1,2022-10-03 10:23:15,10.22 WIB. Kondisi lalin di Simpang Maguwo ter...,https://twitter.com/ATCS_DISHUB_DIY/status/157...,['https://pbs.twimg.com/media/FeHUKPoaEAAV6AE....
2,2022-10-03 08:10:58,08.10WIB Arus lalu lintas simpang MMUGM terpan...,https://twitter.com/ATCS_DISHUB_DIY/status/157...,['https://pbs.twimg.com/media/FeG2CS5aMAAOo2g....
3,2022-10-03 08:10:35,08.10WIB Arus lalu lintas simpang Demangan ter...,https://twitter.com/ATCS_DISHUB_DIY/status/157...,['https://pbs.twimg.com/media/FeG19EQagAEnjNm....
4,2022-10-03 08:10:11,08.09WIB Arus lalu lintas simpang UIN padat,https://twitter.com/ATCS_DISHUB_DIY/status/157...,['https://pbs.twimg.com/media/FeG10TNacAAYXpi....
...,...,...,...,...
115,2022-09-24 15:13:55,15.13 wib arus lalin sp demangan terpantau ram...,https://twitter.com/ATCS_DISHUB_DIY/status/157...,['https://pbs.twimg.com/media/FdaAcHFaAAATBco....
116,2022-09-24 12:56:20,12.55 WIB. Situasi lalin di Simpang Gramedia t...,https://twitter.com/ATCS_DISHUB_DIY/status/157...,['https://pbs.twimg.com/media/FdZg5V4agAAatyK....
117,2022-09-24 12:52:48,12.52 WIB. Situasi lalin di Simpang Condongcat...,https://twitter.com/ATCS_DISHUB_DIY/status/157...,['https://pbs.twimg.com/media/FdZgHE_aUAEklTd....
118,2022-09-24 07:26:14,07.25WIB Kondisi Lalin simpang Gramedia terpan...,https://twitter.com/ATCS_DISHUB_DIY/status/157...,['https://pbs.twimg.com/media/FdYVfXZaMAEPA24....


In [22]:
df.to_excel('TwintFullClean.xlsx', index = False)

In [24]:
dfProcess = pd.read_excel('TwintFullClean.xlsx')
dfStatus = pd.read_excel('DataNonJalan.xlsx')
#dfJalan = pd.read_excel('DataSimpangDIY.xlsx')
dfJalan = pd.read_excel('DataSimpangDIYNormal.xlsx')

listKondisi = dfStatus['Kondisi'].tolist()
listKondisi = [x for x in listKondisi if pd.isnull(x) == False]

listKeterangan = dfStatus['Peristiwa'].tolist()
listKeterangan = [x for x in listKeterangan if pd.isnull(x) == False]

listCuaca = dfStatus['Cuaca'].tolist()
listCuaca = [x for x in listCuaca if pd.isnull(x) == False]

dfProcess = dfProcess[dfProcess['tweet'].str.contains('|'. join(listKondisi))]

print(listKondisi)
print(listKeterangan)
print(listCuaca)

['lancar', 'ramai lancar', 'padat', 'padat merayap', 'tersendat', 'macet']
['kecelakaan', 'tabrakan', 'banjir', 'demonstrasi', 'mogok', 'genangan', 'pohon tumbang']
['berawan', 'hujan', 'gerimis', 'cerah']


In [25]:
timeConv = pd.to_datetime(dfProcess['date'])
timeConv = timeConv.reset_index(drop=True)
dftimeConv = timeConv.dt.strftime('%Y-%b-%d')
dftimeYear = timeConv.dt.strftime('%Y')
dftimeMonth = timeConv.dt.strftime('%B')
dftimeDay = timeConv.dt.day_name()
print(dftimeDay)
print(dftimeMonth)
print(dftimeYear)
print(dftimeConv)
print(timeConv)

0       Monday
1       Monday
2       Monday
3       Monday
4       Monday
        ...   
77    Saturday
78    Saturday
79    Saturday
80    Saturday
81    Saturday
Name: date, Length: 82, dtype: object
0       October
1       October
2       October
3       October
4       October
        ...    
77    September
78    September
79    September
80    September
81    September
Name: date, Length: 82, dtype: object
0     2022
1     2022
2     2022
3     2022
4     2022
      ... 
77    2022
78    2022
79    2022
80    2022
81    2022
Name: date, Length: 82, dtype: object
0     2022-Oct-03
1     2022-Oct-03
2     2022-Oct-03
3     2022-Oct-03
4     2022-Oct-03
         ...     
77    2022-Sep-24
78    2022-Sep-24
79    2022-Sep-24
80    2022-Sep-24
81    2022-Sep-24
Name: date, Length: 82, dtype: object
0    2022-10-03 08:10:58
1    2022-10-03 08:10:35
2    2022-10-03 08:10:11
3    2022-10-03 08:05:19
4    2022-10-03 08:04:45
             ...        
77   2022-09-24 17:16:13
78   2022-09-

In [26]:
# simpangList = dfJalan['simpang'].tolist()
# arahList = dfJalan['arah'].tolist()
simpangList = dfJalan['Simpang'].tolist()
arahList = dfJalan['Arah'].tolist()

simpangList = [x for x in simpangList if pd.isnull(x) == False]
arahList = [x for x in arahList if pd.isnull(x) == False]
streetList = simpangList + arahList
streetList = list(dict.fromkeys(streetList))
print(streetList)

['Tempel', 'Kidsfun', 'Giwangan', 'Gamping', 'MM UGM', 'Demangan', 'Pelem Gurih', 'Blok O', 'Bandara', 'Maguwoharjo', 'Damai Timur', 'Seturan', 'Ngasem', 'Wojo', 'Demak Ijo', 'Jati Kencana', 'UIN', 'Pogung', 'Kentungan', 'Condongcatur', 'Tugu', 'Gramedia', 'Galeria', 'Druwo', 'Kota Gedhe', 'Patran', 'Timur', 'Barat', 'Selatan', 'Utara']


In [27]:
dfInsert = dfProcess.insert(1,"Date", np.nan)
dfInsert = dfProcess.insert(2,"Year", np.nan)
dfInsert = dfProcess.insert(3,"Month", np.nan)
dfInsert = dfProcess.insert(4,"Day", np.nan)
dfInsert = dfProcess.insert(5,"Waktu", np.nan)
dfInsert = dfProcess.insert(6,"Dari", np.nan)
dfInsert = dfProcess.insert(7,"Arah", np.nan)
dfInsert = dfProcess.insert(8,"Kondisi", np.nan)
dfInsert = dfProcess.insert(10,"Hambatan", np.nan)
dfProcess = dfProcess.reset_index(drop=True)
dftimeConv = dftimeConv.reset_index(drop=True)
dfProcess

,date,Date,Year,Month,Day,Waktu,Dari,Arah,Kondisi,tweet,Hambatan,link,photos
0,2022-10-03 08:10:58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,08.10WIB Arus lalu lintas simpang MMUGM terpan...,NaN,https://twitter.com/ATCS_DISHUB_DIY/status/157...,['https://pbs.twimg.com/media/FeG2CS5aMAAOo2g....
1,2022-10-03 08:10:35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,08.10WIB Arus lalu lintas simpang Demangan ter...,NaN,https://twitter.com/ATCS_DISHUB_DIY/status/157...,['https://pbs.twimg.com/media/FeG19EQagAEnjNm....
2,2022-10-03 08:10:11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,08.09WIB Arus lalu lintas simpang UIN padat,NaN,https://twitter.com/ATCS_DISHUB_DIY/status/157...,['https://pbs.twimg.com/media/FeG10TNacAAYXpi....
3,2022-10-03 08:05:19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,08.04WIB Arus lalu lintas simpang Kentungan la...,NaN,https://twitter.com/ATCS_DISHUB_DIY/status/157...,['https://pbs.twimg.com/media/FeG0tiLaAAAvTZQ....
4,2022-10-03 08:04:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,08.04WIB Arus lalu lintas simpang Condongcatur...,NaN,https://twitter.com/ATCS_DISHUB_DIY/status/157...,['https://pbs.twimg.com/media/FeG0lv4aEAA4n51....
...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,2022-09-24 17:16:13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.15WIB Kondisi lalin simpang Galeria lancar,NaN,https://twitter.com/ATCS_DISHUB_DIY/status/157...,['https://pbs.twimg.com/media/FdacdUKaMAAaGMC....
78,2022-09-24 15:15:34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.15 wib lalin sp bandara arah timur terpanta...,NaN,https://twitter.com/ATCS_DISHUB_DIY/status/157...,['https://pbs.twimg.com/media/FdaA55TacAAKOB9....
79,2022-09-24 15:14:57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.14 wib lalin sp gramedia arah timur ramai l...,NaN,https://twitter.com/ATCS_DISHUB_DIY/status/157...,['https://pbs.twimg.com/media/FdaAv-OacAEZfig....
80,2022-09-24 07:26:14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,07.25WIB Kondisi Lalin simpang Gramedia terpan...,NaN,https://twitter.com/ATCS_DISHUB_DIY/status/157...,['https://pbs.twimg.com/media/FdYVfXZaMAEPA24....


In [28]:
tweet = dfProcess['tweet'].tolist()

listDate = []
listTime = []
listFrom = []
listTo = []
listCond = []
listCause = []

for n in range(0, len(tweet), +1):
  newList = tweet[n]
  newStr = newList

  fixedStr = newStr
  strToReplace = {'gangguan mesin': 'mogok', 'demo': 'demonstrasi', 
                  'sp':'simpang', 'disimpang':'simpang', 'berlawanan':'sebaliknya', 'segala arah':'',
                  'kidsfun':'Kidsfun', 'kids fun':'Kidsfun', 'Kids fun':'Kidsfun', 'Kids Fun':'Kidsfun',
                  'jatikencana':'Jati Kencana', 'Jatikencana':'Jati Kencana',
                  'pelemgurih':'Pelem Gurih', 'pelem gurih':'Pelem Gurih', 'Pelem gurih':'Pelem Gurih', 'pelem Gurih':'Pelem Gurih',
                  'blok o':'Blok O', 'blok O':'Blok O', 'Blok o':'Blok O', 'BLOK O':'Blok O', 'BLOK o':'Blok O',
                  'maguwo':'Maguwoharjo', 'Maguwo':'Maguwoharjo', 'MAGUWO':'Maguwoharjo', 'maguwoharjo':'Maguwoharjo', 'MAGUWOHARJO':'Maguwoharjo', 'maguwo harjo':'Maguwoharjo', 'Maguwo harjo':'Maguwoharjo', 'maguwo Harjo':'Maguwoharjo', 'Maguwo Harjo':'Maguwoharjo',
                  'kotagedhe':'Kota Gedhe', 'Kotagedhe':'Kota Gedhe', 'kotaGedhe':'Kota Gedhe', 'KOTAgedhe':'Kota Gedhe', 'kotaGEDHE':'Kota Gedhe', 'kota gedhe':'Kota Gedhe', 'kota Gedhe':'Kota Gedhe', 'KOTA gedhe':'Kota Gedhe', 'kota GEDHE':'Kota Gedhe', ''
                  'mm ugm':'MM UGM', 'MM ugm':'MM UGM', 'mmugm':'MM UGM', 'mm UGM':'MM UGM', 'MMugm':'MM UGM', 'mmUGM':'MM UGM', 'MMUGM':'MM UGM',
                  'tempel':'Tempel', 'giwangan':'Giwangan', 'gamping':'Gamping', 'demangan':'Demangan', 'bandara':'Bandara',
                  'seturan':'Seturan', 'ngasem':'Ngasem', 'wojo':'Wojo', 'uin':'UIN', 'pogung':'Pogung',
                  'kentungan':'Kentungan', 'tugu':'Tugu', 'gramedia':'Gramedia', 'galeria':'Galeria',
                  'condongcatur':'Condongcatur', 'CondongCatur':'Condongcatur',
                  'druwo':'Druwo',
                  'patran':'Patran',
                  'timur':'Timur',
                  'barat':'Barat',
                  'selatan':'Selatan',
                  'utara':'Utara'}
  for key, value in strToReplace.items():
      fixedStr = fixedStr.replace(key, value)
      fixedStr = re.sub("[\(\[].*?[\)\]]", "", fixedStr)

#>>>Proses splitting arah jalan dan splitting waktu<<<
  splitList = fixedStr.split('menuju' and 'arah' or 'ke')
  print(splitList)
  # lenKond = len(splitKond)
  # print(splitKond)

  tweetTime = dfProcess['date'].tolist()
  timeTweet = tweetTime[n]
  splitTime = timeTweet.split(' ')
  #print(splitTime)

  lenList = len(splitList)
  emptyStr = '-'

  tglTweet = dftimeConv[n]
  waktuTweet = splitTime[1]

  #print('Date: ' + tglTweet)
  #print('Tweet Time: ' + waktuTweet)

  print('Tweet: ' + newList)

#>>>Start proses ekstraksi teks dari twitter<<<
  if (lenList > 1):
    fromList = splitList[0]
    fromWord = [word for word in streetList if word in fromList]
    toList = splitList[1]
    toWord = [word for word in streetList if word in toList]
    fromStr = ", ".join(fromWord)
    toStr = ", ".join(toWord)
    print('From: ' + fromStr)
    print('To: ' + toStr)
  else:
    fromList = splitList[0]
    fromWord = [word for word in streetList if word in fromList]
    fromStr = " ".join(fromWord)
    toStr = emptyStr
    print('From: ' + fromStr)
    print('To: ' + toStr)

  tweetKondisi = [x for x in listKondisi if x in newList]
  kondisiStr = " ".join(tweetKondisi)
  if 'ramai lancar' in newList:
    kondisiStr = 'ramai lancar'
  elif 'lancar' in newList:
    kondisiStr = 'lancar'
  kondisiStrrev = emptyStr
  # print('Condition: ' + kondisiStr)

  # if (lenKond > 1):
  #   kondList = splitKond[1]
  #   tweetKondisirev = [x for x in listKondisi if x in kondList]
  #   kondisiStrrev = " ".join(tweetKondisirev)
  #   if 'ramai lancar' in kondList:
  #     kondisiStrrev = 'ramai lancar'
  #   elif 'lancar' in kondList:
  #     kondisiStrrev = 'lancar'
  #   print('Opposite: ' + kondisiStrrev)

  emptyList = []
  tweetKeterangan = [x for x in listKeterangan if x in fixedStr]
  if (tweetKeterangan == emptyList):
    keteranganStr = emptyStr
  else:
    keteranganStr = " ".join(tweetKeterangan)
  #print('Cause: ' + keteranganStr)

  # tweetCuaca = [x for x in listCuaca if x in fixedStr]
  # if (tweetCuaca == emptyList):
  #   cuacaStr = emptyStr
  # else:
  #   cuacaStr = " ".join(tweetCuaca)
  # #print('Weather: ' + cuacaStr)
  # #print('\n')

  listDate.append(tglTweet)
  listTime.append(waktuTweet)
  listFrom.append(fromStr)
  listTo.append(toStr)
  listCond.append(kondisiStr)
  listCause.append(keteranganStr)

#>>>End proses ekstraksi teks dari twitter<<<

#print(listFrom)
print(listTo)
#print(listCond)
#print(listCause)
#print(listWeather)

#>>>Memasukkan hasil proses ekstraksi ke tabel<<<
dfFinalTweet = dfProcess.assign(Date = dftimeConv, Waktu = listTime, Year = dftimeYear, Month = dftimeMonth, Day = dftimeDay, Dari = listFrom, Arah = listTo, Kondisi = listCond, Hambatan = listCause)
dfFinalTweet = dfFinalTweet.drop(['date'], axis=1)
dfFinalTweet = dfFinalTweet.rename(columns = {'tweet':'Tweet'})
print(dfFinalTweet)

['08.10WIB Arus lalu lintas simpang MM UGM terpantau lancar  ']
Tweet: 08.10WIB Arus lalu lintas simpang MMUGM terpantau lancar  
From: MM UGM
To: -
['08.10WIB Arus lalu lintas simpang Demangan terpantau padat  ']
Tweet: 08.10WIB Arus lalu lintas simpang Demangan terpantau padat  
From: Demangan
To: -
['08.09WIB Arus lalu lintas simpang UIN padat  ']
Tweet: 08.09WIB Arus lalu lintas simpang UIN padat  
From: UIN
To: -
['08.04WIB Arus lalu lintas simpang Kentungan lancar  ']
Tweet: 08.04WIB Arus lalu lintas simpang Kentungan lancar  
From: Kentungan
To: -
['08.04WIB Arus lalu lintas simpang Condongcatur padat  ']
Tweet: 08.04WIB Arus lalu lintas simpang Condongcatur padat  
From: Condongcatur
To: -
['08.03WIB Kondisi Lalin simpang Bandara dari kaki Timur padat  ']
Tweet: 08.03WIB Kondisi Lalin simpang Bandara dari kaki timur padat  
From: Bandara Timur
To: -
['08.02WIB Arus lalu lintas simpang Maguwoharjo padat dari semua ', '  ']
Tweet: 08.02WIB Arus lalu lintas simpang Maguwo padat da

In [29]:
dfFinalTweet.to_excel('DatasetTweetFinal.xlsx', index = False)

In [30]:
dfFinalTweet.to_csv('DatasetTweetFinal.csv', index = False)

In [31]:
# read DataFrame
data = pd.read_csv("DatasetTweetFinal.csv")
!mkdir datasets
root = 'datasets'
for (Date), group in data.groupby(['Date']):
     group.to_csv(root + '/' + f'{Date}.csv', index=False)

# print(pd.read_csv("2022-Sep-22.csv"))
# print(pd.read_csv("2022-Sep-17.csv"))

In [93]:
docMaster = []
for file in os.listdir('datasets'):
  if file.endswith ('.csv'):
    docMaster.append (pd.read_csv(os.path.join('datasets', file)))

In [124]:
docMain = pd.concat(docMaster, axis=0)
dfdocMain = docMain.to_csv('DocFull.csv', index = True)
dfdocMain = pd.read_csv('DocFull.csv')
dfdocMain = dfdocMain.dropna(axis = 0)

dfdocMain = dfdocMain.apply(lambda x: x.replace(strToReplace, regex=True))
listdocMain = dfdocMain['Tweet'].tolist()
print(dfdocMain)
print(listdocMain)

# for n in range(0, len(listdocMain), +1):
#   newListdoc = tweet[n]
#   newStrdoc = newListdoc
#   fixedStr = newStrdoc
#   for key, value in strToReplace.items():
#       fixedStr = fixedStr.replace(key, value)
#       fixedStr = re.sub("[\(\[].*?[\)\]]", "", fixedStr)
#       # dfdocMain['Tweet'] = fixedStr
#       listfixedStr = list(fixedStr.split(" "))
#       df = df[df['tweet'].str.contains('|'. join(key_value))]
# print(listfixedStr)
# print(fixedStr)
# print(newListdoc)
# print(listdocMain)

    Unnamed: 0         Date  Year      Month       Day     Waktu  \
1            0  2022-Sep-27  2022  September   Tuesday  18:46:18   
2            1  2022-Sep-27  2022  September   Tuesday  18:45:47   
3            2  2022-Sep-27  2022  September   Tuesday  18:44:07   
4            3  2022-Sep-27  2022  September   Tuesday  18:43:44   
5            4  2022-Sep-27  2022  September   Tuesday  18:43:03   
..         ...          ...   ...        ...       ...       ...   
76           5  2022-Sep-26  2022  September    Monday  14:26:25   
77           0  2022-Oct-01  2022    October  Saturday  08:22:27   
78           1  2022-Oct-01  2022    October  Saturday  08:22:01   
80           3  2022-Oct-01  2022    October  Saturday  07:32:00   
81           4  2022-Oct-01  2022    October  Saturday  07:31:25   

                Dari          Arah       Kondisi  \
1           Demangan  Timur, Utara  ramai lancar   
2                UIN  Timur, Barat  ramai lancar   
3   Maguwoharjoharjo       

In [108]:
docs = doc1 + doc2 + doc3 + doc4 + doc5
arrdocs = np.array(listdocMain)

In [96]:
streetList2 = simpangList + arahList

In [97]:
streetSet = set(streetList2)
arrStreet = np.array(streetList2)

In [98]:
# inverted_index = {}

# for i, streetSet in enumerate(docs):
#   for term in streetSet:
#     if term in inverted_index:
#       inverted_index[term].add(i)
#     else: inverted_index[term] = {i}

# inverted_index

In [114]:
# print(arrdocs)
invIndex = {}
for i in range(len(dfdocMain)):
    check = arrdocs[i]
    for item in arrStreet:
        if item in check:
            if item not in invIndex:
                invIndex[item] = []
            if item in invIndex:
                invIndex[item].append(i+1)

invIndex

{'Demangan': [1, 6, 12, 21, 44, 48],
 'Timur': [1,
  2,
  4,
  5,
  6,
  7,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  19,
  25,
  26,
  27,
  28,
  30,
  32,
  35,
  36,
  37,
  39,
  41,
  42,
  43,
  45,
  46,
  47,
  50,
  52,
  53,
  57,
  58,
  68,
  69,
  70],
 'Selatan': [1, 7, 11, 12, 16, 37, 42, 43, 44, 50, 55, 70],
 'Utara': [1, 12, 16, 17, 26, 37, 42, 43, 44, 45, 52, 55],
 'UIN': [2, 14, 22, 27, 28, 39, 46, 50, 53],
 'Barat': [2,
  3,
  5,
  8,
  11,
  13,
  14,
  15,
  17,
  18,
  26,
  27,
  28,
  29,
  31,
  35,
  36,
  37,
  39,
  40,
  42,
  43,
  45,
  46,
  50,
  52,
  53,
  58,
  67,
  69,
  70],
 'Maguwoharjo': [3, 18, 29, 31, 40, 67],
 'Bandara': [4, 19, 25, 30, 32, 41, 47, 57, 68],
 'Gramedia': [5, 8, 10, 13, 36, 43, 58, 59, 63, 69],
 'Galeria': [7, 56],
 'Pogung': [9, 54, 64],
 'Seturan': [11, 42, 49, 62, 70],
 'Jati Kencana': [15, 33, 35],
 'MM UGM': [16, 20, 38, 51, 55, 60, 65],
 'Condongcatur': [17, 24, 37, 61],
 'Kentungan': [23],
 'Tugu': [26, 45, 52, 66],

In [100]:
# posting_list = inverted_index['Seturan']
# posting_list

In [101]:
# import os
# path = 'datasets'
# filenames = os.listdir(path)
# # print(filenames)

# def is_name_in_csv(word,csv_file):
#   with open(csv_file,"r") as f:
#     data = f.read()
#   return bool(re.search(word, data))

# list_file_csv = filenames
# word = ('|'. join(streetList))
# for csv_file in list_file_csv:
#   if is_name_in_csv(word,csv_file):
#     print(f"Found {word} in {csv_file}")
#   else:
#     print(f"Finished. Can't find any more data with that word")
#     break

In [111]:
from collections import defaultdict
inv_index = defaultdict(list)
for doc, words in zip(
        dfdocMain['Date'],
        dfdocMain['Dari'].str.findall('|'. join(listFrom)).map(set)):
    for word in words:
        inv_index[word].append(doc)
print(inv_index)

defaultdict(<class 'list'>, {'': ['2022-Sep-27', '2022-Sep-27', '2022-Sep-27', '2022-Sep-27', '2022-Sep-27', '2022-Sep-27', '2022-Sep-27', '2022-Sep-30', '2022-Sep-30', '2022-Sep-30', '2022-Sep-30', '2022-Sep-30', '2022-Sep-30', '2022-Sep-30', '2022-Sep-30', '2022-Sep-30', '2022-Sep-30', '2022-Sep-30', '2022-Sep-30', '2022-Oct-03', '2022-Oct-03', '2022-Oct-03', '2022-Oct-03', '2022-Oct-03', '2022-Oct-03', '2022-Sep-25', '2022-Sep-25', '2022-Sep-25', '2022-Sep-25', '2022-Sep-25', '2022-Sep-25', '2022-Sep-25', '2022-Sep-25', '2022-Sep-25', '2022-Sep-28', '2022-Sep-28', '2022-Sep-28', '2022-Sep-28', '2022-Sep-28', '2022-Sep-28', '2022-Sep-28', '2022-Sep-28', '2022-Sep-28', '2022-Sep-28', '2022-Sep-28', '2022-Sep-28', '2022-Sep-28', '2022-Sep-29', '2022-Sep-29', '2022-Sep-29', '2022-Sep-29', '2022-Sep-24', '2022-Sep-24', '2022-Sep-24', '2022-Sep-24', '2022-Sep-24', '2022-Sep-24', '2022-Sep-24', '2022-Sep-24', '2022-Sep-24', '2022-Sep-26', '2022-Sep-26', '2022-Sep-26', '2022-Sep-26', '2022-

In [103]:
# print(tweet)
# arr_tweet = np.array(tweet)
# arr_streetList = np.array(streetList)
# print(arr_streetList)
# #print(arr_tweet)
# dict = {}

# for i in range(len(tweet)):
#     check = arr_tweet[i].lower()
#     for item in arr_streetList:
#         if item in check:
#             if item not in dict:
#                 dict[item] = []
#             if item in dict:
#                 dict[item].append(i+1)

# dict

In [126]:
dfFinish = pd.read_excel('DatasetTweetFinal.xlsx')
dfFinish = dfFinish.dropna(axis=0)
fixedListFrom = list(dict.fromkeys(listFrom))
# print(fixedListFrom)
greet = "Hi! Saya akan mengambil tweet Jakarta Utara dari TMCPoldaMetroJaya. Coba tulis nama jalannya disini ya!"
print(greet)

def posindexing(inputstr):
  from collections import defaultdict
  ind = inputstr
  pos_index = defaultdict(list)
  for doc, words in zip(
      dfdocMain['Date'],
      dfdocMain['Dari'].str.findall(ind).map(set)):
    for word in words:
        pos_index[word].append(doc)
  print(pos_index)

def indexing(inputstr):
  invIndex = {}
  for i in range(len(dfdocMain)):
      check = arrdocs[i]
      for item in arrStreet:
          if item in check:
              if item not in invIndex:
                  invIndex[item] = []
              if item in invIndex:
                  invIndex[item].append(i+1)
  x = invIndex[inputstr]
  print(x)

def start():
  print('Coba masukkan nama jalan ')
  inputstr = input('')
  checkWord = [word for word in fixedListFrom if word in inputstr]
  fixedInput = ''.join(checkWord)
  print(checkWord)
  dfBot = dfFinish[dfFinish['Dari'].str.contains(inputstr)]
  posindexing(inputstr)
  indexing(inputstr)
  print('\n')

  showDate = dfBot['Date'].tolist()
  showYear = dfBot['Year'].tolist()
  showMonth = dfBot['Month'].tolist()
  showDay = dfBot['Day'].tolist()
  showTime = dfBot['Waktu'].tolist()
  showTweet = dfBot['Tweet'].tolist()
  showFrom = dfBot['Dari'].tolist()
  showTo = dfBot['Arah'].tolist()
  showCond = dfBot['Kondisi'].tolist()
  showCause = dfBot['Hambatan'].tolist()
  showAll = dfBot.values.tolist()
  showAll = sum(showAll, [])
  showAll = ' '.join(map(str, showAll))

  if inputstr in showAll:
    print("Yay! Saya menemukan beberapa Tweet!\n")
    for n in range(0, len(showTweet), +1):
      print('Date: ' + str(showDate[n]))
      print('Year: ' + str(showYear[n]))
      print('Month: ' + showMonth[n])
      print('Day: ' + showDay[n])
      print('Time: ' + showTime[n])
      print('Tweet: ' + showTweet[n])
      print('From: ' + showFrom[n])
      print('To: ' + showTo[n])
      print('Condition: ' + showCond[n])
      print('Cause: ' + str(showCause[n]))

      print('\n')

  else:
    print('Hmm... Sepertinya TMC Polda Metro Jaya belum Tweet jalan itu. Bagaimana dengan jalan lainnya?')
    start()
start()

Hi! Saya akan mengambil tweet Jakarta Utara dari TMCPoldaMetroJaya. Coba tulis nama jalannya disini ya!
Coba masukkan nama jalan 
Seturan
['', 'Seturan']
defaultdict(<class 'list'>, {'Seturan': ['2022-Sep-30', '2022-Sep-28', '2022-Sep-29', '2022-Sep-26', '2022-Oct-01']})
[11, 42, 49, 62, 70]


Yay! Saya menemukan beberapa Tweet!

Date: 2022-Oct-01
Year: 2022
Month: October
Day: Saturday
Time: 07:31:25
Tweet: 07.31 wib lalin sp seturan arah timur dan barat ramai lancar arah selatan terpantau lancar  
From: Seturan
To: Timur, Barat
Condition: ramai lancar
Cause: -


Date: 2022-Sep-30
Year: 2022
Month: September
Day: Friday
Time: 09:08:25
Tweet: 09.07 wib lalin sp seturan arah timur dan barat ramai lancar arah selatan terpantau lancar  
From: Seturan
To: Timur, Barat
Condition: ramai lancar
Cause: -


Date: 2022-Sep-29
Year: 2022
Month: September
Day: Thursday
Time: 16:25:53
Tweet: 16.25WIB Kondisi Lalin simpang Seturan terpantau padat  
From: Seturan
To: -
Condition: padat
Cause: -


Dat